In [ ]:
# default_exp utils

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# utils

> Various utility functions used by the blurr package.

In [ ]:
#export
import sys, inspect
from enum import Enum

import pandas as pd
import torch

from transformers import *
from fastai2.text.all import *

In [ ]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


In [ ]:
#export
def str_to_class(classname):
    "converts string representation to class"
    return getattr(sys.modules[__name__], classname)

In [ ]:
#export
class Singleton:
    def __init__(self,cls):
        self._cls, self._instance = cls, None

    def __call__(self, *args, **kwargs):
        if self._instance == None: self._instance = self._cls(*args, **kwargs)
        return self._instance

`Singleton` functions as python decorator.  Use this above any class to turn that class into a singleton (see [here](https://python-3-patterns-idioms-test.readthedocs.io/en/latest/Singleton.html) for more info on the singleton pattern).

In [ ]:
@Singleton
class TestSingleton: pass

a = TestSingleton()
b = TestSingleton()
test_eq(a,b)

## ModelHelper

In [ ]:
#export
@Singleton
class ModelHelper():
    
    def __init__(self):
        # get hf classes (tokenizers, configs, models, etc...)
        transformer_classes = inspect.getmembers(sys.modules[__name__], 
                                                 lambda member: inspect.isclass(member)
                                                 and member.__module__.startswith('transformers.'))
        
        # build a df that we can query against to get various transformers objects/info
        self._df = pd.DataFrame(transformer_classes, columns=['class_name', 'class_location'])
        
        # add the module each class is included in
        self._df['module'] = self._df.class_location.apply(lambda v: v.__module__)
        
        # remove class_location (don't need it anymore)
        self._df.drop(labels=['class_location'], axis=1, inplace=True)
        
        # break up the module into separate cols
        module_parts_df = self._df.module.str.split(".", n = -1, expand = True) 
        for i in range(len(module_parts_df.columns)):
            self._df[f'module_part_{i}'] = module_parts_df[i]

        # using module part 1, break up the functional area and arch into separate cols
        module_part_1_df = self._df.module_part_1.str.split("_", n = 1, expand = True) 
        self._df[['functional_area', 'arch']] = module_part_1_df
        
        # if functional area = modeling, pull out the task it is built for
        model_type_df = self._df[(self._df.functional_area == 'modeling')].class_name.str.split('For', n=1, expand=True)
        
        model_type_df[1] = np.where(model_type_df[1].notnull(), 
                                    'For' + model_type_df[1].astype(str), 
                                    model_type_df[1])
        
        self._df['model_task'] = model_type_df[1]
        self._df['model_task'] = self._df['model_task'].str.replace('For', '', n=1, case=True, regex=False)
        
        model_type_df = self._df[(self._df.functional_area == 'modeling')].class_name.str.split('With', n=1, expand=True)
        model_type_df[1] = np.where(model_type_df[1].notnull(), 
                                    'With' + model_type_df[1].astype(str), 
                                    self._df[(self._df.functional_area == 'modeling')].model_task)
        
        self._df['model_task'] = model_type_df[1]
        self._df['model_task'] = self._df['model_task'].str.replace('With', '', n=1, case=True, regex=False)
        
        # look at what we're going to remove (use to verify we're just getting rid of stuff we want too)
        # df[~df['hf_class_type'].isin(['modeling', 'configuration', 'tokenization'])]
        
        # only need these 3 functional areas for our querying purposes
        self._df = self._df[self._df['functional_area'].isin(['modeling', 'configuration', 'tokenization'])]
        
    def get_architectures(self): 
        """Used to get all the architectures supported by your `Transformers` install"""
        return sorted(self._df[(self._df.arch.notna()) & 
                        (self._df.arch != None) & 
                        (self._df.arch != 'utils')].arch.unique().tolist())
    
    def get_config(self, arch): 
        """Used the locate the name of the configuration class for a given architecture"""
        config = self._df[(self._df.functional_area == 'configuration') & 
                          (self._df.arch == arch)].class_name.values[0]
        
        return str_to_class(config)
    
    def get_tokenizers(self, arch): 
        """Used to get the available huggingface tokenizers for a given architecture. Note: There may be 
        multiple tokenizers and so this returns a list.
        """
        toks = sorted(self._df[(self._df.functional_area == 'tokenization') & 
                               (self._df.arch == arch)].class_name.values)
        
        return [str_to_class(tok_name) for tok_name in toks]
    
    def get_tasks(self, arch=None): 
        """Get the type of tasks for which there is a custom model for (*optional: by architecture*). 
        There are a number of customized models built for specific tasks like token classification, 
        question/answering, LM, etc....
        """
        query = ['model_task.notna()']
        if (arch): query.append(f'arch == "{arch}"')

        return sorted(self._df.query(' & '.join(query), engine='python').model_task.unique().tolist())
    
    def get_models(self, arch=None, task=None):
        """The transformer models available for use (optional: by architecture | task)"""
        query = ['functional_area == "modeling"']
        if (arch): query.append(f'arch == "{arch}"')
        if (task): query.append(f'model_task == "{task}"')

        models = sorted(self._df.query(' & '.join(query)).class_name.tolist())
        return [str_to_class(model_name) for model_name in models] 
    
    def get_classes_for_model(self, model_name_or_cls):
        """Get tokenizers, config, and model for a given model name / class"""
        model_name = model_name_or_cls if isinstance(model_name_or_cls, str) else model_name_or_cls.__name__

        meta = self._df[self._df.class_name == model_name]
        tokenizers = self.get_tokenizers(meta.arch.values[0])
        config = self.get_config(meta.arch.values[0])

        return (config, tokenizers, str_to_class(model_name))
    
    def get_model_architecture(self, model_name_or_enum):
        """Get the architecture for a given model name / enum"""
        model_name = model_name_or_enum if isinstance(model_name_or_enum, str) else model_name_or_enum.name
        return self._df[self._df.class_name == model_name].arch.values[0]
    
    def get_hf_objects(self, pretrained_model_name_or_path, task=None,
                       config=None, tokenizer_cls=None, model_cls=None, 
                       config_kwargs={}, tokenizer_kwargs={}, model_kwargs={}, cache_dir=None):
        """Returns the architecture (str), config (obj), tokenizer (obj), and model (obj) given at minimum a
        `pre-trained model name or path`. Specify a `task` to ensure the right "AutoModelFor<task>" is used to
        create the model.
        
        Optionally, you can pass a config (obj), tokenizer (class), and/or model (class) (along with any 
        related kwargs for each) to get as specific as you want w/r/t what huggingface objects are returned.
        """
        
        # config
        if (config is None):
            config = AutoConfig.from_pretrained(pretrained_model_name_or_path, cache_dir=cache_dir, **config_kwargs)
            
        # tokenizer
        if (tokenizer_cls is None):
            tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path, 
                                                      cache_dir=cache_dir, 
                                                      **tokenizer_kwargs)
        else:
            tokenizer = tokenizer_cls.from_pretrained(pretrained_model_name_or_path, 
                                                      cache_dir=cache_dir, 
                                                      **tokenizer_kwargs)
            
        # model
        if (model_cls is None and task is None):
            model = AutoModel.from_pretrained(pretrained_model_name_or_path, 
                                              config=config, 
                                              cache_dir=cache_dir, 
                                              **model_kwargs)
        else:
            if (model_cls is None and task is not None): 
                model_cls = self.get_models(arch="auto", task=task.name)[0]
            
            model = model_cls.from_pretrained(pretrained_model_name_or_path, 
                                              config=config, 
                                              cache_dir=cache_dir, 
                                              **model_kwargs)
            
        #arch
        arch = self.get_model_architecture(type(model).__name__)
        
        return (arch, config, tokenizer, model)

`ModelHelper` is a `Singleton` (there exists only one instance, and the same instance is returned upon subsequent instantiation requests).  You can get at via the `BLURR_MODEL_HELPER` constant below.

In [ ]:
mh = ModelHelper()
mh2 = ModelHelper()
test_eq(mh, mh2)

In [ ]:
#hide
display_df(mh._df.head())

print(list(mh._df.model_task.unique()))
print('')
print(list(mh._df.functional_area.unique()))
print('')
print(list(mh._df.module_part_2.unique()))
print('')
print(list(mh._df.module_part_3.unique()))

,class_name,module,module_part_0,module_part_1,module_part_2,module_part_3,functional_area,arch,model_task
1,AdaptiveEmbedding,transformers.modeling_transfo_xl,transformers,modeling_transfo_xl,None,None,modeling,transfo_xl,None
2,AlbertConfig,transformers.configuration_albert,transformers,configuration_albert,None,None,configuration,albert,NaN
3,AlbertForMaskedLM,transformers.modeling_albert,transformers,modeling_albert,None,None,modeling,albert,MaskedLM
4,AlbertForMultipleChoice,transformers.modeling_albert,transformers,modeling_albert,None,None,modeling,albert,MultipleChoice
5,AlbertForPreTraining,transformers.modeling_albert,transformers,modeling_albert,None,None,modeling,albert,PreTraining


[None, nan, 'MaskedLM', 'MultipleChoice', 'PreTraining', 'QuestionAnswering', 'SequenceClassification', 'TokenClassification', 'CausalLM', 'Seq2SeqLM', 'LMHead', 'ConditionalGeneration', 'NextSentencePrediction', 'QuestionAnsweringSimple', 'LMHeadModel', 'Classification']

['modeling', 'configuration', 'tokenization']

[None]

[None]


### Provide global helper constant

Users of this library can simply use `BLURR_MODEL_HELPER` to access all the `ModelHelper` capabilities without having to fetch an instance themselves.

In [ ]:
#export
BLURR_MODEL_HELPER = ModelHelper()

In [ ]:
show_doc(ModelHelper(ModelHelper).get_architectures)

<h4 id="ModelHelper.get_architectures" class="doc_header"><code>ModelHelper.get_architectures</code><a href="__main__.py#L53" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelHelper.get_architectures</code>()

Used to get all the architectures supported by your `Transformers` install

In [ ]:
print(BLURR_MODEL_HELPER.get_architectures())

['albert', 'auto', 'bart', 'bert', 'bert_japanese', 'camembert', 'ctrl', 'distilbert', 'electra', 'encoder_decoder', 'flaubert', 'gpt2', 'longformer', 'marian', 'mmbt', 'mobilebert', 'openai', 'reformer', 'retribert', 'roberta', 't5', 'transfo_xl', 'utils_base', 'utils_fast', 'xlm', 'xlm_roberta', 'xlnet']


We'll also create an enum for downstream tasks

In [ ]:
#export
HF_ARCHITECTURES = Enum('HF_ARCHITECTURES', BLURR_MODEL_HELPER.get_architectures())

In [ ]:
print(L(HF_ARCHITECTURES))

(#27) [<HF_ARCHITECTURES.albert: 1>,<HF_ARCHITECTURES.auto: 2>,<HF_ARCHITECTURES.bart: 3>,<HF_ARCHITECTURES.bert: 4>,<HF_ARCHITECTURES.bert_japanese: 5>,<HF_ARCHITECTURES.camembert: 6>,<HF_ARCHITECTURES.ctrl: 7>,<HF_ARCHITECTURES.distilbert: 8>,<HF_ARCHITECTURES.electra: 9>,<HF_ARCHITECTURES.encoder_decoder: 10>...]


In [ ]:
show_doc(ModelHelper(ModelHelper).get_config)

<h4 id="ModelHelper.get_config" class="doc_header"><code>ModelHelper.get_config</code><a href="__main__.py#L59" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelHelper.get_config</code>(**`arch`**)

Used the locate the name of the configuration class for a given architecture

In [ ]:
print(BLURR_MODEL_HELPER.get_config('bert'))

<class 'transformers.configuration_bert.BertConfig'>


In [ ]:
show_doc(ModelHelper(ModelHelper).get_tokenizers)

<h4 id="ModelHelper.get_tokenizers" class="doc_header"><code>ModelHelper.get_tokenizers</code><a href="__main__.py#L66" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelHelper.get_tokenizers</code>(**`arch`**)

Used to get the available huggingface tokenizers for a given architecture. Note: There may be 
multiple tokenizers and so this returns a list.

In [ ]:
print(BLURR_MODEL_HELPER.get_tokenizers('electra'))

[<class 'transformers.tokenization_electra.ElectraTokenizer'>, <class 'transformers.tokenization_electra.ElectraTokenizerFast'>]


In [ ]:
show_doc(ModelHelper(ModelHelper).get_tasks)

<h4 id="ModelHelper.get_tasks" class="doc_header"><code>ModelHelper.get_tasks</code><a href="__main__.py#L75" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelHelper.get_tasks</code>(**`arch`**=*`None`*)

Get the type of tasks for which there is a custom model for (*optional: by architecture*). 
There are a number of customized models built for specific tasks like token classification, 
question/answering, LM, etc....

In [ ]:
print(BLURR_MODEL_HELPER.get_tasks())
print('')
print(BLURR_MODEL_HELPER.get_tasks('bart'))

['CausalLM', 'Classification', 'ConditionalGeneration', 'LMHead', 'LMHeadModel', 'MaskedLM', 'MultipleChoice', 'NextSentencePrediction', 'PreTraining', 'QuestionAnswering', 'QuestionAnsweringSimple', 'Seq2SeqLM', 'SequenceClassification', 'TokenClassification']

['ConditionalGeneration', 'QuestionAnswering', 'SequenceClassification']


We'll create an enum for tasks as well, one for all tasks and another for tasks available via huggingface's `AutoModel` capabilities

In [ ]:
#export
HF_TASKS_ALL = Enum('HF_TASKS_ALL', BLURR_MODEL_HELPER.get_tasks())
HF_TASKS_AUTO = Enum('HF_TASKS_AUTO', BLURR_MODEL_HELPER.get_tasks('auto'))

In [ ]:
print('--- all tasks ---')
print(L(HF_TASKS_ALL))
print('\n--- auto only ---')
print(L(HF_TASKS_AUTO))

--- all tasks ---
(#14) [<HF_TASKS_ALL.CausalLM: 1>,<HF_TASKS_ALL.Classification: 2>,<HF_TASKS_ALL.ConditionalGeneration: 3>,<HF_TASKS_ALL.LMHead: 4>,<HF_TASKS_ALL.LMHeadModel: 5>,<HF_TASKS_ALL.MaskedLM: 6>,<HF_TASKS_ALL.MultipleChoice: 7>,<HF_TASKS_ALL.NextSentencePrediction: 8>,<HF_TASKS_ALL.PreTraining: 9>,<HF_TASKS_ALL.QuestionAnswering: 10>...]

--- auto only ---
(#9) [<HF_TASKS_AUTO.CausalLM: 1>,<HF_TASKS_AUTO.LMHead: 2>,<HF_TASKS_AUTO.MaskedLM: 3>,<HF_TASKS_AUTO.MultipleChoice: 4>,<HF_TASKS_AUTO.PreTraining: 5>,<HF_TASKS_AUTO.QuestionAnswering: 6>,<HF_TASKS_AUTO.Seq2SeqLM: 7>,<HF_TASKS_AUTO.SequenceClassification: 8>,<HF_TASKS_AUTO.TokenClassification: 9>]


In [ ]:
HF_TASKS_ALL.Classification

<HF_TASKS_ALL.Classification: 2>

In [ ]:
show_doc(ModelHelper(ModelHelper).get_models)

<h4 id="ModelHelper.get_models" class="doc_header"><code>ModelHelper.get_models</code><a href="__main__.py#L85" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelHelper.get_models</code>(**`arch`**=*`None`*, **`task`**=*`None`*)

The transformer models available for use (optional: by architecture | task)

In [ ]:
print(L(BLURR_MODEL_HELPER.get_models()))

(#136) [<class 'transformers.modeling_transfo_xl.AdaptiveEmbedding'>,<class 'transformers.modeling_albert.AlbertForMaskedLM'>,<class 'transformers.modeling_albert.AlbertForMultipleChoice'>,<class 'transformers.modeling_albert.AlbertForPreTraining'>,<class 'transformers.modeling_albert.AlbertForQuestionAnswering'>,<class 'transformers.modeling_albert.AlbertForSequenceClassification'>,<class 'transformers.modeling_albert.AlbertForTokenClassification'>,<class 'transformers.modeling_albert.AlbertModel'>,<class 'transformers.modeling_albert.AlbertPreTrainedModel'>,<class 'transformers.modeling_auto.AutoModel'>...]


In [ ]:
print(BLURR_MODEL_HELPER.get_models(arch='bert'))

[<class 'transformers.modeling_bert.BertForMaskedLM'>, <class 'transformers.modeling_bert.BertForMultipleChoice'>, <class 'transformers.modeling_bert.BertForNextSentencePrediction'>, <class 'transformers.modeling_bert.BertForPreTraining'>, <class 'transformers.modeling_bert.BertForQuestionAnswering'>, <class 'transformers.modeling_bert.BertForSequenceClassification'>, <class 'transformers.modeling_bert.BertForTokenClassification'>, <class 'transformers.modeling_bert.BertLMHeadModel'>, <class 'transformers.modeling_bert.BertLayer'>, <class 'transformers.modeling_bert.BertModel'>, <class 'transformers.modeling_bert.BertPreTrainedModel'>]


In [ ]:
print(BLURR_MODEL_HELPER.get_models(task='TokenClassification'))

[<class 'transformers.modeling_albert.AlbertForTokenClassification'>, <class 'transformers.modeling_auto.AutoModelForTokenClassification'>, <class 'transformers.modeling_bert.BertForTokenClassification'>, <class 'transformers.modeling_camembert.CamembertForTokenClassification'>, <class 'transformers.modeling_distilbert.DistilBertForTokenClassification'>, <class 'transformers.modeling_electra.ElectraForTokenClassification'>, <class 'transformers.modeling_longformer.LongformerForTokenClassification'>, <class 'transformers.modeling_mobilebert.MobileBertForTokenClassification'>, <class 'transformers.modeling_roberta.RobertaForTokenClassification'>, <class 'transformers.modeling_xlm.XLMForTokenClassification'>, <class 'transformers.modeling_xlm_roberta.XLMRobertaForTokenClassification'>, <class 'transformers.modeling_xlnet.XLNetForTokenClassification'>]


In [ ]:
print(BLURR_MODEL_HELPER.get_models(arch='bert', task='TokenClassification'))

[<class 'transformers.modeling_bert.BertForTokenClassification'>]


In [ ]:
show_doc(ModelHelper(ModelHelper).get_classes_for_model)

<h4 id="ModelHelper.get_classes_for_model" class="doc_header"><code>ModelHelper.get_classes_for_model</code><a href="__main__.py#L94" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelHelper.get_classes_for_model</code>(**`model_name_or_cls`**)

Get tokenizers, config, and model for a given model name / class

In [ ]:
config, tokenizers, model = BLURR_MODEL_HELPER.get_classes_for_model('RobertaForSequenceClassification')

print(config)
print(tokenizers[0])
print(model)

<class 'transformers.configuration_roberta.RobertaConfig'>
<class 'transformers.tokenization_roberta.RobertaTokenizer'>
<class 'transformers.modeling_roberta.RobertaForSequenceClassification'>


In [ ]:
config, tokenizers, model = BLURR_MODEL_HELPER.get_classes_for_model(DistilBertModel)

print(config)
print(tokenizers[0])
print(model)

<class 'transformers.configuration_distilbert.DistilBertConfig'>
<class 'transformers.tokenization_distilbert.DistilBertTokenizer'>
<class 'transformers.modeling_distilbert.DistilBertModel'>


In [ ]:
show_doc(ModelHelper(ModelHelper).get_model_architecture)

<h4 id="ModelHelper.get_model_architecture" class="doc_header"><code>ModelHelper.get_model_architecture</code><a href="__main__.py#L104" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelHelper.get_model_architecture</code>(**`model_name_or_enum`**)

Get the architecture for a given model name / enum

In [ ]:
BLURR_MODEL_HELPER.get_model_architecture('RobertaForSequenceClassification')

'roberta'

### Methods for loading pre-trained (configs, tokenizer, model) hugginface classes

In [ ]:
show_doc(ModelHelper(ModelHelper).get_hf_objects)

<h4 id="ModelHelper.get_hf_objects" class="doc_header"><code>ModelHelper.get_hf_objects</code><a href="__main__.py#L109" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelHelper.get_hf_objects</code>(**`pretrained_model_name_or_path`**, **`task`**=*`None`*, **`config`**=*`None`*, **`tokenizer_cls`**=*`None`*, **`model_cls`**=*`None`*, **`config_kwargs`**=*`{}`*, **`tokenizer_kwargs`**=*`{}`*, **`model_kwargs`**=*`{}`*, **`cache_dir`**=*`None`*)

Returns the architecture (str), config (obj), tokenizer (obj), and model (obj) given at minimum a
`pre-trained model name or path`. Specify a `task` to ensure the right "AutoModelFor<task>" is used to
create the model.

Optionally, you can pass a config (obj), tokenizer (class), and/or model (class) (along with any 
related kwargs for each) to get as specific as you want w/r/t what huggingface objects are returned.

In [ ]:
arch, config, tokenizer, model = BLURR_MODEL_HELPER.get_hf_objects("bert-base-cased-finetuned-mrpc",
                                                                   task=HF_TASKS_AUTO.LMHead)

print(arch)
print(type(config))
print(type(tokenizer))
print(type(model))

/home/wgilliam/anaconda3/envs/blurr/lib/python3.7/site-packages/transformers/modeling_auto.py:798: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of the model checkpoint at bert-base-cased-finetuned-mrpc were not used when initializing BertForMaskedLM: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificat

bert
<class 'transformers.configuration_bert.BertConfig'>
<class 'transformers.tokenization_bert.BertTokenizer'>
<class 'transformers.modeling_bert.BertForMaskedLM'>


In [ ]:
arch, tokenizer, config, model = BLURR_MODEL_HELPER.get_hf_objects("fmikaelian/flaubert-base-uncased-squad",
                                                                   task=HF_TASKS_AUTO.QuestionAnswering)

print(arch)
print(type(config))
print(type(tokenizer))
print(type(model))

Some weights of the model checkpoint at fmikaelian/flaubert-base-uncased-squad were not used when initializing FlaubertForQuestionAnsweringSimple: ['qa_outputs.start_logits.dense.weight', 'qa_outputs.start_logits.dense.bias', 'qa_outputs.end_logits.dense_0.weight', 'qa_outputs.end_logits.dense_0.bias', 'qa_outputs.end_logits.LayerNorm.weight', 'qa_outputs.end_logits.LayerNorm.bias', 'qa_outputs.end_logits.dense_1.weight', 'qa_outputs.end_logits.dense_1.bias', 'qa_outputs.answer_class.dense_0.weight', 'qa_outputs.answer_class.dense_0.bias', 'qa_outputs.answer_class.dense_1.weight']
- This IS expected if you are initializing FlaubertForQuestionAnsweringSimple from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing FlaubertForQuestionAnsweringSimple from the checkpoint of a model that you expect to be exactly identical (initi

flaubert
<class 'transformers.tokenization_flaubert.FlaubertTokenizer'>
<class 'transformers.configuration_flaubert.FlaubertConfig'>
<class 'transformers.modeling_flaubert.FlaubertForQuestionAnsweringSimple'>


In [ ]:
arch, tokenizer, config, model = BLURR_MODEL_HELPER.get_hf_objects("bert-base-cased-finetuned-mrpc",
                                                                   config=None,
                                                                   tokenizer_cls=BertTokenizer, 
                                                                   model_cls=BertForNextSentencePrediction)
print(arch)
print(type(config))
print(type(tokenizer))
print(type(model))

Some weights of the model checkpoint at bert-base-cased-finetuned-mrpc were not used when initializing BertForNextSentencePrediction: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForNextSentencePrediction were not initialized from the model checkpoint at bert-base-cased-finetuned-mrpc and are newly initialized: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert
<class 'transformers.tokenization_bert.BertTokenizer'>
<class 'transformers.configuration_bert.BertConfig'>
<class 'transformers.modeling_bert.BertForNextSentencePrediction'>


## Cleanup

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01a_data-language-modeling.ipynb.
Converted 01c_data-question-answering.ipynb.
Converted 01d_data-token-classification.ipynb.
Converted 01e_data-text-generation.ipynb.
Converted 02_modeling-core.ipynb.
Converted 02_training-summarization.ipynb.
Converted 02a_modeling-language-modeling.ipynb.
Converted 02c_modeling-question-answering.ipynb.
Converted 02d_modeling-token-classification.ipynb.
Converted 02e_modeling-text-generation.ipynb.
Converted index.ipynb.
